# Linear Regression Health Costs Calculator

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

## Uploading the libraries

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

## Getting the data

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

## Body of a program

In [ ]:
dataset.describe()

In [ ]:
dataset.describe(include = 'object')

In [ ]:
dataset['sex'].unique()

In [ ]:
dataset['smoker'].unique()

In [ ]:
dataset['region'].unique()

## Data cleaning

In [ ]:
dataset.isna().sum()

## Convert categorical data

In [ ]:
dataset['sex'].replace(['female', 'male'],
                        [1, 0], inplace=True)

In [ ]:
dataset['smoker'].replace(['yes', 'no'],
                        [1, 0], inplace=True)

In [ ]:
dataset['region'].replace(['southwest', 'southeast', 'northwest', 'northeast'],
                        [3, 2, 1, 0], inplace=True)

In [ ]:
dataset.head()

## Or we can use pd.get_dummies

In [ ]:
dataset = pd.get_dummies(dataset)

In [ ]:
dataset.head()

## Train & test split

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=8)
test_dataset = dataset.drop(train_dataset.index)

## Data inspection

In [ ]:
sns.pairplot(train_dataset[['age', 'bmi',	'children',	'expenses']], diag_kind='kde')

## Split features from expenses

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
train_dataset.head()

## Normalization

In [ ]:
def normalization(x):
  return (x - x.mean()) / x.std()
train_dataset = normalization(train_dataset)
test_dataset = normalization(test_dataset)

In [ ]:
train_dataset.head()

## Building model №1

In [ ]:
def build_model():
  model = keras.Sequential([
      layers.Dense(256, activation = 'relu', input_shape=[len(train_dataset.keys())]),
      layers.Dense(256, activation = 'relu'),
      layers.Dense(128, activation = 'relu'),
      layers.Dense(1)
  ])

  model.compile(loss = 'mae',
                optimizer='adam',
                 metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

## Training model

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 300)

history = model.fit(train_dataset, train_labels, epochs = 1000, verbose = 2, callbacks=[early_stop])

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(
    history.history['mae']
)
plt.ylabel('MAE')
plt.xlabel('Epochs')
plt.grid(True)
plt.show()

## Assessment

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
